# Sequence to Sequence model for english to tamil transilation using rnn encoder and decoder ( at character level)

## dataset is downloaded from http://www.manythings.org/anki/

In [15]:
from __future__ import print_function

from keras.models import Model,load_model
from keras.layers import Input,LSTM,Dense
from keras.callbacks import ModelCheckpoint
import numpy as np

batch_size=64
epochs=25
latent_dim=256
data_path="/home/santhosh/keras/rnn/rnn_encoder_decoder/data/tam.txt"

## vectorizing the data

In [16]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines:
    line=line.split('\t')
    if(len(line)!=2):
        continue
    input_text,target_text=line
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text='\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
input_characters=sorted(list(input_characters))
targer_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_len=max([len(text) for text in input_texts])
max_decoder_seq_len=max([len(text) for text in target_texts])

print('Number of samples:',len(input_texts))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)


Number of samples: 180
number of unique input token: 53
number of unique output token: 54
Max Sequence length for inputs: 94
Max Sequence length for outputs: 111


## defining token2index

In [17]:
input_token2index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token2index=dict([(char,i) for i,char in enumerate(target_characters)])

## defing encoder_input,decoder_input and decoder_output

In [18]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_tokens),dtype='float32')

## creating training dataset

In [6]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token2index[char]]=1
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token2index[char]]=1
        # decoder_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i,t-1,target_token2index[char]]=1


## Define an input sequence and process it.

In [7]:


encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




## Set up the decoder, using encoder_states as initial state.

In [8]:


decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



## Define the model that will turn
## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

In [9]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])

## function to test model

In [10]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token2index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token2index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token2index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


## Run training

In [14]:

iteration=0

# load weights
print('loading the weights')
model=load_model('char_level.h5')

# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
print("Testing Samples\n"+"-"*50)
for i in range(5):
    index=np.random.randint(len(input_texts))
    encoded_input_sequence=encoder_input_data[index: index + 1]
    output_sequence=decode_sequence(encoded_input_sequence)
    print(input_texts[index],output_sequence)
print("-"*50)

iteration_file="/home/santhosh/keras/rnn/rnn_encoder_decoder/iteration_char_level.txt"
try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    #print(iteration)
    file.close()
    
except:
    print('no file exist')

# checkpoint
filepath="weights_best_char_level.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    print('Iteration:',iteration+1)
    #training
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,callbacks=callbacks_list)
    #prepare sample_data to test 5 samples:
    print("-"*50)
    for i in range(5):
        index=np.random.randint(len(input_texts))
        encoded_input_sequence=encoder_input_data[index: index + 1]
        output_sequence=decode_sequence(encoded_input_sequence)
        print(input_texts[index],output_sequence)
    print("-"*50)
    # Save model
    file=open(iteration_file,'a')
    file.write('iteration:'+str(iteration+1)+'\n')
    file.close()
    iteration+=1
    model.save('char_level.h5')

no file exist
Iteration: 1
Train on 144 samples, validate on 36 samples
Epoch 1/25
144/144 [==============================] - 7s 46ms/step - loss: 0.7078 - acc: 0.8024 - val_loss: 1.2087 - val_acc: 0.6829
Epoch 2/25


/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


144/144 [==============================] - 7s 47ms/step - loss: 0.7040 - acc: 0.4950 - val_loss: 1.2146 - val_acc: 0.6762
Epoch 3/25
144/144 [==============================] - 7s 49ms/step - loss: 0.6941 - acc: 0.8037 - val_loss: 1.1943 - val_acc: 0.6897
Epoch 4/25
144/144 [==============================] - 7s 47ms/step - loss: 0.6882 - acc: 0.8108 - val_loss: 1.1677 - val_acc: 0.6794
Epoch 5/25
144/144 [==============================] - 6s 45ms/step - loss: 0.6900 - acc: 0.7140 - val_loss: 1.1791 - val_acc: 0.6794
Epoch 6/25
144/144 [==============================] - 7s 48ms/step - loss: 0.6767 - acc: 0.4929 - val_loss: 1.1689 - val_acc: 0.6847
Epoch 7/25
144/144 [==============================] - 7s 47ms/step - loss: 0.6741 - acc: 0.8072 - val_loss: 1.1682 - val_acc: 0.6899
Epoch 8/25
144/144 [==============================] - 7s 48ms/step - loss: 0.6658 - acc: 0.7367 - val_loss: 1.1683 - val_acc: 0.6839
Epoch 9/25
144/144 [==============================] - 7s 46ms/step - loss: 0.657

144/144 [==============================] - 7s 48ms/step - loss: 0.4643 - acc: 0.4535 - val_loss: 0.9640 - val_acc: 0.5991
Epoch 2/25
144/144 [==============================] - 7s 48ms/step - loss: 0.4702 - acc: 0.6846 - val_loss: 0.9920 - val_acc: 0.2080
Epoch 3/25
144/144 [==============================] - 7s 48ms/step - loss: 0.4767 - acc: 0.2418 - val_loss: 0.9525 - val_acc: 0.2082
Epoch 4/25
144/144 [==============================] - 7s 48ms/step - loss: 0.4618 - acc: 0.3507 - val_loss: 0.9576 - val_acc: 0.4162
Epoch 5/25
144/144 [==============================] - 7s 45ms/step - loss: 0.4592 - acc: 0.3141 - val_loss: 0.9643 - val_acc: 0.1406
Epoch 6/25
144/144 [==============================] - 6s 45ms/step - loss: 0.4542 - acc: 0.1530 - val_loss: 0.9634 - val_acc: 0.1436
Epoch 7/25
144/144 [==============================] - 7s 47ms/step - loss: 0.4511 - acc: 0.4313 - val_loss: 0.9357 - val_acc: 0.1727
Epoch 8/25
144/144 [==============================] - 7s 47ms/step - loss: 0.456

144/144 [==============================] - 7s 48ms/step - loss: 0.3339 - acc: 0.2554 - val_loss: 0.9515 - val_acc: 0.2718
Epoch 2/25
144/144 [==============================] - 7s 48ms/step - loss: 0.3214 - acc: 0.3133 - val_loss: 0.9618 - val_acc: 0.2457
Epoch 3/25
144/144 [==============================] - 7s 45ms/step - loss: 0.3221 - acc: 0.2589 - val_loss: 0.9685 - val_acc: 0.2142
Epoch 4/25
144/144 [==============================] - 7s 47ms/step - loss: 0.3245 - acc: 0.2279 - val_loss: 0.9742 - val_acc: 0.2307
Epoch 5/25
144/144 [==============================] - 7s 48ms/step - loss: 0.3210 - acc: 0.2401 - val_loss: 0.9454 - val_acc: 0.2220
Epoch 6/25
144/144 [==============================] - 7s 47ms/step - loss: 0.3123 - acc: 0.2489 - val_loss: 0.9640 - val_acc: 0.1652
Epoch 7/25
144/144 [==============================] - 7s 47ms/step - loss: 0.3107 - acc: 0.2282 - val_loss: 0.9656 - val_acc: 0.1922
Epoch 8/25
144/144 [==============================] - 7s 48ms/step - loss: 0.300

144/144 [==============================] - 7s 47ms/step - loss: 0.1934 - acc: 0.2916 - val_loss: 1.0653 - val_acc: 0.1877
Epoch 2/25
144/144 [==============================] - 7s 45ms/step - loss: 0.1758 - acc: 0.2911 - val_loss: 1.0898 - val_acc: 0.2270
Epoch 3/25
144/144 [==============================] - 7s 50ms/step - loss: 0.1791 - acc: 0.3116 - val_loss: 1.1017 - val_acc: 0.2032
Epoch 4/25
144/144 [==============================] - 7s 45ms/step - loss: 0.1729 - acc: 0.3017 - val_loss: 1.0889 - val_acc: 0.2047
Epoch 5/25
144/144 [==============================] - 6s 44ms/step - loss: 0.1731 - acc: 0.2855 - val_loss: 1.0961 - val_acc: 0.1782
Epoch 6/25
144/144 [==============================] - 7s 50ms/step - loss: 0.1759 - acc: 0.2726 - val_loss: 1.1176 - val_acc: 0.2945
Epoch 7/25
144/144 [==============================] - 8s 58ms/step - loss: 0.1753 - acc: 0.3155 - val_loss: 1.1026 - val_acc: 0.1774
Epoch 8/25
144/144 [==============================] - 8s 58ms/step - loss: 0.171

Iteration: 9
Train on 144 samples, validate on 36 samples
Epoch 1/25
144/144 [==============================] - 6s 42ms/step - loss: 0.1109 - acc: 0.3174 - val_loss: 1.2707 - val_acc: 0.1847
Epoch 2/25
144/144 [==============================] - 7s 47ms/step - loss: 0.1174 - acc: 0.3024 - val_loss: 1.2681 - val_acc: 0.1962
Epoch 3/25
144/144 [==============================] - 6s 44ms/step - loss: 0.1020 - acc: 0.3175 - val_loss: 1.2651 - val_acc: 0.2277
Epoch 4/25
144/144 [==============================] - 7s 49ms/step - loss: 0.1007 - acc: 0.3308 - val_loss: 1.2819 - val_acc: 0.2150
Epoch 5/25
144/144 [==============================] - 5s 31ms/step - loss: 0.1042 - acc: 0.3148 - val_loss: 1.2745 - val_acc: 0.1832
Epoch 6/25
144/144 [==============================] - 4s 30ms/step - loss: 0.1045 - acc: 0.3242 - val_loss: 1.2674 - val_acc: 0.1904
Epoch 7/25
144/144 [==============================] - 8s 55ms/step - loss: 0.1000 - acc: 0.3032 - val_loss: 1.2699 - val_acc: 0.1882
Epoch 8/25


He let go of the rope. ிளல!)ு)??ழஜஜஜஜஜCஜஜC(உஉ. )C2CஇனோCோCஇCஇ னனல!கன00ல!குன00லக00்்	ிளளூூலக00லக00லக00்்	ிளளூூலக00லக00லக00்்	ிளளூூலக00லக0
--------------------------------------------------
Iteration: 11
Train on 144 samples, validate on 36 samples
Epoch 1/25
144/144 [==============================] - 8s 59ms/step - loss: 0.0673 - acc: 0.3300 - val_loss: 1.4469 - val_acc: 0.2092
Epoch 2/25
144/144 [==============================] - 8s 58ms/step - loss: 0.0660 - acc: 0.3193 - val_loss: 1.4518 - val_acc: 0.2092
Epoch 3/25
144/144 [==============================] - 8s 57ms/step - loss: 0.0662 - acc: 0.3277 - val_loss: 1.4671 - val_acc: 0.2080
Epoch 4/25
144/144 [==============================] - 8s 56ms/step - loss: 0.0664 - acc: 0.3306 - val_loss: 1.4593 - val_acc: 0.1869
Epoch 5/25
144/144 [==============================] - 8s 55ms/step - loss: 0.0628 - acc: 0.3196 - val_loss: 1.4648 - val_acc: 0.1984
Epoch 6/25
144/144 [==============================] - 8s 55ms/step - loss: 0.0654 - acc: 

KeyboardInterrupt: 